<a href="https://colab.research.google.com/github/Mkps/nvidia_workshop/blob/main/Profiling_CUDA_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Jupyter notebook is a web based environment for interactive computing. It is capable of running code in a wide variety of programming languages. The notebook consists of cells which can be of type code or markdown (by default, new cells are created as code cells). You can execute the content of code cells by clicking the Run button. Cells which are actively running code will show a 'Play' icon and cells which have completed will show a green tick.

**Connect to GPU runtime**: If using Google Colab, please ensure you have this notebook enabled with GPU before running (Runtime->Change runtime type). If you are not using Google Colab you can ignore this instruction.

<img src="https://drive.google.com/uc?export&id=1rroMUCXRqssxsUNo29qbkeYjqy792BsE"/>

# Installing NVIDIA Nsight System and Compute

In [1]:
# Installing nsight-systems-2022.4.1 & nsight-compute-2022.1.1
!apt-get update -y && \
     DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
         apt-transport-https \
         ca-certificates \
         gnupg \
         wget && \
     rm -rf /var/lib/apt/lists/*
!wget -qO - https://developer.download.nvidia.com/devtools/repos/ubuntu2004/amd64/nvidia.pub | apt-key add - && \
     echo "deb https://developer.download.nvidia.com/devtools/repos/ubuntu2004/amd64/ /" >> /etc/apt/sources.list.d/nsight.list && \
     apt-get update -y && \
     DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
         nsight-systems-2022.1.1 nsight-compute-2022.1.1 && \
     rm -rf /var/lib/apt/lists/*

# setting environment variable path
import os
os.environ["PATH"] = "/usr/local/bin" + os.pathsep + os.getenv("PATH")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,267 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,343 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-securit

Let's check the installed version of the NVIDIA profiler tools to ensure the installation was successfull.

In [2]:
!nsys --version
!echo "---"
!ncu --version

NVIDIA Nsight Systems version 2022.1.1.61-1d07dc0
---
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2022 NVIDIA Corporation
Version 2022.3.0.0 (build 31729285) (public-release)


The expected output is

```
NVIDIA Nsight Systems version 2022.1.1.61-1d07dc0
---
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2022 NVIDIA Corporation
Version 2022.1.1.0 (build 30914944) (public-release)
```
If your output is different, make sure you are connected to the runtime and re-run installation.

In [3]:
# download the source code
import gdown
import os
url = 'https://drive.google.com/uc?id=1FDsXtSVmqk8TI4tZvAMbYiS8761_gWLM&export=download'
#url = 'https://drive.google.com/u/0/uc?export=download&confirm=1FDsXtSVmqk8TI4tZvAMbYiS8761_gWLM'
output = '/home/code'
gdown.download(url, output, quiet=False,proxy=None)
!unzip /home/code -d /home
!rm /home/code

Downloading...
From: https://drive.google.com/uc?id=1FDsXtSVmqk8TI4tZvAMbYiS8761_gWLM&export=download
To: /home/code
100%|██████████| 173k/173k [00:00<00:00, 97.2MB/s]

Archive:  /home/code
   creating: /home/source_code/
   creating: /home/source_code/lab1/
  inflating: /home/source_code/lab1/Makefile  
  inflating: /home/source_code/lab1/miniWeather_serial.cpp  
   creating: /home/source_code/lab1/.ipynb_checkpoints/
  inflating: /home/source_code/lab1/.ipynb_checkpoints/Makefile-checkpoint  
  inflating: /home/source_code/lab1/.ipynb_checkpoints/miniWeather_serial-checkpoint.cpp  
  inflating: /home/source_code/lab1/.ipynb_checkpoints/miniWeather_serial-checkpoint.f90  
  inflating: /home/source_code/lab1/miniWeather_serial.f90  
   creating: /home/source_code/lab2/
  inflating: /home/source_code/lab2/Makefile  
   creating: /home/source_code/lab2/.ipynb_checkpoints/
  inflating: /home/source_code/lab2/.ipynb_checkpoints/Makefile-checkpoint  
  inflating: /home/source_code/lab2/.ipynb_checkpoints/miniWeather_openacc-checkpoint.cpp  
  inflating: /home/source_code/lab2/.ipynb_checkpoints/miniWeather_openacc-checkpoint.f90  
  inflating: /home/source

Let' check we downloaded the source files.

In [4]:
!ls /home/source_code

lab1  lab2  lab3  lab4	lab5  lab6


The expected output is:

```
lab1  lab2  lab3  lab4	lab5  lab6
```

If your output is different, make sure you are connected to the runtime and re-run the cell to download the source code.

Before we begin, let's execute the below cell to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by clicking on the play button which appears when hovering over the cell with the mouse will run the code and you should see the output displayed beneath the code cell.

In [5]:
!nvidia-smi

Tue Oct 10 10:15:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Lab 6 [Optional]



###  Learning objectives
The **goal** of this lab is to:

- Learn how to profile an application using NVIDIA Nsight™ Systems and Nsight Compute (CUDA example)

We do not intend to cover:

- How to use the CUDA programming model
- How to implement shared memory
- Optimization methods

**NOTE: All the below screenshots are from an NVIDIA A100 Tensor Core GPU.**

Please read the below **Introduction notebook** before you start.

In this lab, we will be porting the serial Jacobi code to the GPU using CUDA and obtaining the fastest performance possible. To achieve this, we need to consider Amdahl's law (please see the section on [Amdahl's Law and Scaling](profiling_lab1.ipynb#amdahls)) and remove as many performance limitations  as possible. Common performance limiters are:

- Serial portion of the code on the CPU
- Memory movements (Device To/From Host)
- Latency as a result of launching GPU kernels
- Not enough work to hide instruction latency
- Non-efficient memory access pattern
    - Uncoalesced memory accesses, lack of cache reuse, not using shared memory (Read more at [the NVIDIA Technical Blog](https://developer.nvidia.com/blog/boosting-application-performance-with-gpu-memory-prefetching/)) and [CUDA C Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/)
- Low arithmetic intensity (the ratio between compute work [FLOPs] and data movement [bytes])

### Step 0: Analyze the Code

In this lab, we will be working on the Jacobi iteration code that iteratively converges to the correct value by computing new values at each point from the average of neighboring  points.

<!--<img src="images/jacobi_formula.png" width="90%">-->
<img src="https://drive.google.com/uc?export=download&id=1w1tqEKAM0DyUmnQhUNMl2deW_lZrHGro">


**Understand and analyze** the code:

In [6]:
!cat /home/source_code/lab6/jacobi.cpp

// Copyright (c) 2022 NVIDIA Corporation.  All rights reserved.

#include <iostream>
#include <iomanip>
#include <ctime>
#include <cmath>
#include <limits>

#define N 64

#define IDX(i, j) ((j) + (i) * N)

void allocate_memory (float** f, float** f_old, float** error) {
    *f = (float*) malloc(N * N * sizeof(float));
    *f_old = (float*) malloc(N * N * sizeof(float));
    *error = (float*) malloc(sizeof(float));
}

void free_memory (float* f, float* f_old, float* error) {
    free(f);
    free(f_old);
    free(error);
}

void initialize_data (float* f) {
    // Set up simple sinusoidal boundary conditions
    for (int j = 0; j < N; ++j) {
        for (int i = 0; i < N; ++i) {

            if (i == 0 || i == N-1) {
                f[IDX(i,j)] = sin(j * 2 * M_PI / (N - 1));
            }
            else if (j == 0 || j == N-1) {
                f[IDX(i,j)] = sin(i * 2 * M_PI / (N - 1));
            }
            else {
                f[IDX(i,j)] = 0.0f;
            }

        }
    }

To obtain the best performance from the GPU and utilize the hardware, one should follow the cyclical process (analyze, parallelize, optimize). Start by compiling the code and running it on the CPU (**Note:** error will be printed periodically as output).

In [7]:
!cd /home/source_code/lab6 && make

nvcc -o jacobi jacobi.cpp  


### Step 1: Instrument the Code to Find Bottlenecks

Before we start  porting to the GPU, the first step is to identify the compute expensive part of the application to find  the performance bottlenecks. One way to do this would be to use the CPU walltime to measure different parts of the code. Another method would be adding NVIDIA Tools Extension software development kit (NVTX). We added the NVTX application programming interface (API) to the code.

**Understand and analyze** the code:

In [8]:
!cat /home/source_code/lab6/jacobi_step1.cpp

// Copyright (c) 2022 NVIDIA Corporation.  All rights reserved.

#include <iostream>
#include <iomanip>
#include <cmath>
#include <limits>
#include <ctime>
#include <nvToolsExt.h>

#define N 64

#define IDX(i, j) ((j) + (i) * N)

void allocate_memory (float** f, float** f_old, float** error) {
    *f = (float*) malloc(N * N * sizeof(float));
    *f_old = (float*) malloc(N * N * sizeof(float));
    *error = (float*) malloc(sizeof(float));
}

void free_memory (float* f, float* f_old, float* error) {
    free(f);
    free(f_old);
    free(error);
}

void initialize_data (float* f) {
    // Set up simple sinusoidal boundary conditions
    for (int j = 0; j < N; ++j) {
        for (int i = 0; i < N; ++i) {

            if (i == 0 || i == N-1) {
                f[IDX(i,j)] = sin(j * 2 * M_PI / (N - 1));
            }
            else if (j == 0 || j == N-1) {
                f[IDX(i,j)] = sin(i * 2 * M_PI / (N - 1));
            }
            else {
                f[IDX(i,j)] = 0.0f;
      

Now, compile the code and **profile** with `nsys` (Reminder: `--stats=true` shows the summary output that includes information about time spent in the various NVTX regions.) (Reminder: we need to link it against the right runtime library (`libnvToolsExt.so`))

In [9]:
!cd /home/source_code/lab6 && make jacobi_step1 && nsys profile --stats=true -o jacobi_step1 -f true ./jacobi_step1

nvcc -o jacobi_step1 -x cu -lnvToolsExt -lineinfo jacobi_step1.cpp
Error after iteration 0 = 0.0438475
Error after iteration 25 = 0.0032022
Error after iteration 50 = 0.00181223
Error after iteration 75 = 0.00126184
Error after iteration 100 = 0.000957702
Error after iteration 125 = 0.000762262
Error after iteration 150 = 0.000625323
Error after iteration 175 = 0.000523902
Error after iteration 200 = 0.000445912
Error after iteration 225 = 0.000384343
Error after iteration 250 = 0.000334796
Error after iteration 275 = 0.000294325
Error after iteration 300 = 0.000260843
Error after iteration 325 = 0.00023282
Error after iteration 350 = 0.000209104
Error after iteration 375 = 0.000188812
Error after iteration 400 = 0.00017127
Error after iteration 425 = 0.000155952
Error after iteration 450 = 0.000142454
Error after iteration 475 = 0.000130462
Error after iteration 500 = 0.000119732
Error after iteration 525 = 0.000110069
Error after iteration 550 = 0.000101323
Success!
Run time = 0.0276

Download the report by running the below cell and open the report via the Nsight Systems UI.

In [10]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step1.nsys-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The timeline of the application is as shown below (feel free to hover your mouse over each section to see more detail; for example, the thread state).

<!--<img src="images/jacobi_1.png" width="90%">-->
<img src="https://drive.google.com/uc?export=download&id=1yKoXC__BnIyQpR7qO5-hs-VIALMBmnDF">

### Step 2: Using Unified Memory

[Unified Memory(UM)](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/) is a single memory address space accessible from any processor in a system. It allows applications to allocate data that can be read or written from code running on either CPUs or GPUs. Now, to allocate Unified Memory, we replace calls to `malloc()` with calls to `cudaMallocManaged()` without making any other changes to the rest of the code.


**Understand and analyze** the code:

In [11]:
!cat /home/source_code/lab6/jacobi_step2.cpp

// Copyright (c) 2022 NVIDIA Corporation.  All rights reserved.

#include <iostream>
#include <iomanip>
#include <cmath>
#include <limits>
#include <ctime>
#include <nvToolsExt.h>

#define N 64

#define IDX(i, j) ((j) + (i) * N)

// error checking macro
#define cudaCheckErrors(msg)                                    \
    do {                                                        \
        cudaError_t __err = cudaGetLastError();                 \
        if (__err != cudaSuccess) {                             \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n",  \
                    msg, cudaGetErrorString(__err),             \
                    __FILE__, __LINE__);                        \
            fprintf(stderr, "*** FAILED - ABORTING\n");         \
            exit(1);                                            \
        }                                                       \
    } while (0)

void allocate_memory (float** f, float** f_old, float** error) {
    

Now, compile and profile the code.

In [12]:
!cd /home/source_code/lab6 && make jacobi_step2 && nsys profile --stats=true -o jacobi_step2 -f true ./jacobi_step2

nvcc -o jacobi_step2 -x cu -lnvToolsExt -lineinfo jacobi_step2.cpp
Error after iteration 0 = 0.0438475
Error after iteration 25 = 0.0032022
Error after iteration 50 = 0.00181223
Error after iteration 75 = 0.00126184
Error after iteration 100 = 0.000957702
Error after iteration 125 = 0.000762262
Error after iteration 150 = 0.000625323
Error after iteration 175 = 0.000523902
Error after iteration 200 = 0.000445912
Error after iteration 225 = 0.000384343
Error after iteration 250 = 0.000334796
Error after iteration 275 = 0.000294325
Error after iteration 300 = 0.000260843
Error after iteration 325 = 0.00023282
Error after iteration 350 = 0.000209104
Error after iteration 375 = 0.000188812
Error after iteration 400 = 0.00017127
Error after iteration 425 = 0.000155952
Error after iteration 450 = 0.000142454
Error after iteration 475 = 0.000130462
Error after iteration 500 = 0.000119732
Error after iteration 525 = 0.000110069
Error after iteration 550 = 0.000101323
Success!
Run time = 0.5275

Download the report by running the below cell and open the report via the Nsight Systems user interface (UI).

In [13]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step2.nsys-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The timeline of the application is shown below.

<!--<img src="images/jacobi_2.png" width="90%">-->
<img src="https://drive.google.com/uc?export=download&id=185mvFtRq6dGl5Ijw6UIFzLLNBLPlhjpT">


The application runs much longer than before. As you can see from the profiler report, there is a cost of initializing CUDA that can be high. In this example, this cost is much higher than the actual calculation time.

<!--<img src="images/jacobi_2_.png" width="90%">-->
<img src="https://drive.google.com/uc?export=download&id=1xVH8j9RAaFbIBs7xOnByYonn5jpMymMj">


### Step 3: Make the Problem Bigger

If we try to reduce the cost of computation and add fast kernels, it will still be slower than the CPU-only version (without UM). Like the previous lab, we simply need to make the problem bigger by either adding more particles/elements or increasing the iteration count. In this example, we increase the grid points `N` which achieves a finer spatial resolution and is more accurate. This will make the Jacobi relaxation step a big chunk of the total application runtime.


**Understand and analyze** the code:

In [14]:
!cat /home/source_code/lab6/jacobi_step3.cpp

// Copyright (c) 2022 NVIDIA Corporation.  All rights reserved.

#include <iostream>
#include <iomanip>
#include <cmath>
#include <limits>
#include <ctime>
#include <nvToolsExt.h>

#define N 2048

#define IDX(i, j) ((j) + (i) * N)

// error checking macro
#define cudaCheckErrors(msg)                                    \
    do {                                                        \
        cudaError_t __err = cudaGetLastError();                 \
        if (__err != cudaSuccess) {                             \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n",  \
                    msg, cudaGetErrorString(__err),             \
                    __FILE__, __LINE__);                        \
            fprintf(stderr, "*** FAILED - ABORTING\n");         \
            exit(1);                                            \
        }                                                       \
    } while (0)

void allocate_memory (float** f, float** f_old, float** error) {
  

Now, compile and profile the code.

In [15]:
!cd /home/source_code/lab6 && make jacobi_step3 && nsys profile --stats=true -o jacobi_step3 -f true ./jacobi_step3

nvcc -o jacobi_step3 -x cu -lnvToolsExt -lineinfo jacobi_step3.cpp
Error after iteration 0 = 0.00781058
Error after iteration 25 = 0.000608073
Error after iteration 50 = 0.000366189
Error after iteration 75 = 0.000271313
Error after iteration 100 = 0.00021912
Error after iteration 125 = 0.000185581
Error after iteration 150 = 0.000161997
Error after iteration 175 = 0.000144382
Error after iteration 200 = 0.000130678
Error after iteration 225 = 0.000119665
Error after iteration 250 = 0.000110599
Error after iteration 275 = 0.000102984
Success!
Run time = 72.36 seconds
Generating '/tmp/nsys-report-120e.qdstrm'
[1/8] [========================100%] jacobi_step3.nsys-rep
Importer error status: Importation succeeded with non-fatal errors.
**** Analysis failed with:
Status: TargetProfilingFailed
Props {
  Items {
    Type: DeviceId
    Value: "Local (CLI)"
  }
}
Error {
  Type: RuntimeError
  SubError {
    Type: ProcessEventsError
    Props {
      Items {
        Type: ErrorText
        Val

Download the report by running the below cell and open the report via the Nsight Systems user interface (UI).

In [16]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step3.nsys-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



With a close look at the "Timeline View", we can see that the Jacobi relaxation step is now a big portion of the total application runtime.

<!--<img src="images/jacobi_3.png" width="90%">-->
<img src="https://drive.google.com/uc?export=download&id=14UwH05ktYoGU-eWsgWKyEtUre4vHC90u">

You can also see the most expensive portion of the application from the profiler report. Right-click on the NVTX row from the "Timline View", then choose "Show in Events View". From the box at the bottom, you can sort all the ranges by duration. Now, you can see the `jacobi_step()` (Jacobi relaxation step) is the most compute expensive part of the code.

<!--<img src="images/jacobi_3_event.png" width="90%">-->
<img src="https://drive.google.com/uc?export=download&id=1K512a45AABoS-HahiMRrHSJJtC4uk2Pu">


### Step 4: Port the `jacobi_step` Kernel to GPU using CUDA

We convert the `jacobi_step()` to a CUDA kernel, parallelizing over the inner and outer loop using a 2D threadblock of size `32x32`. Except for the `error`, each part can be updated independently `atomicadd()` to perform that.

**Understand and analyze** the code:

In [ ]:
!cat /home/source_code/lab6/jacobi_step4.cpp

Now, compile and profile the code.

In [25]:
!cd /home/source_code/lab6 && make jacobi_step4 && nsys profile --stats=true -o jacobi_step4 -f true ./jacobi_step4

make: 'jacobi_step4' is up to date.
Error after iteration 0 = 0.00781058
Error after iteration 25 = 0.000608074
Error after iteration 50 = 0.000366187
Error after iteration 75 = 0.000271309
Error after iteration 100 = 0.000219121
Error after iteration 125 = 0.000185579
Error after iteration 150 = 0.000161997
Error after iteration 175 = 0.000144383
Error after iteration 200 = 0.000130679
Error after iteration 225 = 0.000119665
Error after iteration 250 = 0.0001106
Error after iteration 275 = 0.000102985
Success!
Run time = 46.07 seconds
Generating '/tmp/nsys-report-bb6c.qdstrm'
[1/8] [========================100%] jacobi_step4.nsys-rep
Importer error status: Importation succeeded with non-fatal errors.
**** Analysis failed with:
Status: TargetProfilingFailed
Props {
  Items {
    Type: DeviceId
    Value: "Local (CLI)"
  }
}
Error {
  Type: RuntimeError
  SubError {
    Type: ProcessEventsError
    Props {
      Items {
        Type: ErrorText
        Value: "/build/agent/work/20a3cfcd1

Download the report by running the below cell and open the report via the Nsight Systems user interface (UI).

In [ ]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step4.nsys-rep')

Let's check the NVTX ranges in the "Events View". We can see that the "Jacobi Step" is significantly faster but the "Swap data" is slower than before.

<!--<img src="images/jacobi_4_.png" width="40%">-->
<img src="https://drive.google.com/uc?export=download&id=1K74vlMuj1MZ016UQWmR4qbpTvSBT-NxX">

If we look at the CUDA row, and zoom in, we see all those gaps that are marked as "Swap data" regions on the NVTX row. There is a lot of data movement from host to device in `jacobi_step()` kernel and device to host in the `swap_data()` function (due to the Unified Memory usage).

<!--<img src="images/jacobi_4_1.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1xyy36iJH9hHruhNkeFAH9QL4ATvM_WRX">


### Step 5: Port the `swap_data` Kernel to GPU using CUDA

We port the `swap_data()` function to the GPU using CUDA to do most of the computation on the device and reduce the data movement by keeping most of the data on the GPU.

**Understand and analyze** the code:

In [26]:
!cat /home/source_code/lab6/jacobi_step5.cpp

// Copyright (c) 2022 NVIDIA Corporation.  All rights reserved.

#include <iostream>
#include <iomanip>
#include <cmath>
#include <limits>
#include <ctime>
#include <nvToolsExt.h>

#define N 2048

#define IDX(i, j) ((j) + (i) * N)

// error checking macro
#define cudaCheckErrors(msg)                                    \
    do {                                                        \
        cudaError_t __err = cudaGetLastError();                 \
        if (__err != cudaSuccess) {                             \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n",  \
                    msg, cudaGetErrorString(__err),             \
                    __FILE__, __LINE__);                        \
            fprintf(stderr, "*** FAILED - ABORTING\n");         \
            exit(1);                                            \
        }                                                       \
    } while (0)

void allocate_memory (float** f, float** f_old, float** error) {
  

Now, compile and profile the code.


In [27]:
!cd /home/source_code/lab6 && make jacobi_step5 && nsys profile --stats=true -o jacobi_step5 -f true ./jacobi_step5

nvcc -o jacobi_step5 -x cu -lnvToolsExt -lineinfo jacobi_step5.cpp
Error after iteration 0 = 0.00781058
Error after iteration 25 = 0.000608074
Error after iteration 50 = 0.000366185
Error after iteration 75 = 0.00027131
Error after iteration 100 = 0.000219121
Error after iteration 125 = 0.000185579
Error after iteration 150 = 0.000161997
Error after iteration 175 = 0.000144383
Error after iteration 200 = 0.000130679
Error after iteration 225 = 0.000119665
Error after iteration 250 = 0.000110599
Error after iteration 275 = 0.000102984
Success!
Run time = 3.323 seconds
Generating '/tmp/nsys-report-ea9e.qdstrm'
[1/8] [========================100%] jacobi_step5.nsys-rep
Importer error status: Importation succeeded with non-fatal errors.
**** Analysis failed with:
Status: TargetProfilingFailed
Props {
  Items {
    Type: DeviceId
    Value: "Local (CLI)"
  }
}
Error {
  Type: RuntimeError
  SubError {
    Type: ProcessEventsError
    Props {
      Items {
        Type: ErrorText
        Val

Download the report by running the below cell and open the report via the Nsight Systems user interface (UI).

In [ ]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step5.nsys-rep')

Let's check the NVTX ranges in the "Events View". We can see that the application is much faster now ("Swap data" is now faster than before).

<!--<img src="images/jacobi_5_0.png" width="40%">-->
<img src="https://drive.google.com/uc?export=download&id=1GvO6tXO8ERMjUAEMKumZk6cm5O9hgP0i">

The timeline of the application is shown below.

<!--<img src="images/jacobi_5_1.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1Z0DbsVO-etdHsyy98nX_3SNbrim_oWFg">


By looking at the CUDA row, it is clear that most of the application runtime is now spent in kernels and the data movement (green rectangles) is also reduced. Let's dig deeper and analyze the kernels in Nsight Compute to see if there are any other optimizations we can apply.


### Step 6: Analyze and Improve the Kernels

In this section, we want to find out if the kernel is compute-bound, memory-bound, or latency-bound. To ensure the kernel is not limited by latency, we need to expose enough work to keep a large number of threads running on the GPU. In this example, `N` is equal to 2048, which means the problem size is `2048 x 2048`. Since the order of magnitude number of threads a modern GPU can run simultaneously is O(100k), then we probably have enough work to keep the device busy.

In the case of memory bound or compute bound, we need to think of arithmetic intensity which is the ratio between compute work (FLOPs) and data movement (bytes) expressed as Floating Point Operations per second (FLOP/S). If this ratio is an order 10, then it would be compute bound.

```cpp
f[IDX(i,j)] = 0.25f * (f_old[IDX(i+1,j)] + f_old[IDX(i-1,j)] +
                      f_old[IDX(i,j+1)] + f_old[IDX(i,j-1)]);
```

The `jacobi_step()` kernel computes four floating point operations (three adds and one multiply) and moves four words (each 4 bytes for single precision floating point), so the arithmetic intensity is `4 (FLOPs) / 16 (bytes) = 0.25 FLOPs / byte` and it seems to be in the memory bandwidth region.

Now, profile the kernel with Nsight Compute to verify our hypotheses.

We only profile one invocation of the kernel since it has similar performance characteristics, and to allow the device to warmp up we skip the first few.

In [28]:
!cd /home/source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:jacobi -f -o jacobi_step5_0 --set full ./jacobi_step5

==PROF== Connected to process 12354 (/home/source_code/lab6/jacobi_step5)
Error after iteration 0 = 0.00781058
==PROF== Profiling "jacobi_step": 0%....50%....100% - 31 passes
Error after iteration 25 = 0.000608073
Error after iteration 50 = 0.000366185
Error after iteration 75 = 0.00027131
Error after iteration 100 = 0.000219121
Error after iteration 125 = 0.00018558
Error after iteration 150 = 0.000161997
Error after iteration 175 = 0.000144383
Error after iteration 200 = 0.000130677
Error after iteration 225 = 0.000119665
Error after iteration 250 = 0.000110599
Error after iteration 275 = 0.000102984
Success!
Run time = 4.443 seconds
==PROF== Disconnected from process 12354
==PROF== Report: /home/source_code/lab6/jacobi_step5_0.ncu-rep


Download the report by running the below cell and open the report via the Nsight Compute user interface (UI).

In [29]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step5_0.ncu-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The "Speed of Light Throughput" section gives a high-level overview of the throughput for the compute and memory resources of the GPU for each unit. Based on this information, we can find the performance limiters and catagorize them into four possible combinations:

- Compute Bound: SM>50% & Mem<50%
- Bandwidth Bound: SM<50% & Mem>50%
- Latency Bound: SM<50% & Mem<50%
- Compute and Bandwidth Bound : SM>50% & Mem>50%

According to the *GPU Speed of Light Throughput* section (see screenshot below), the kernel exhibits low compute throughput and memory bandwidth utilization relative to the peak performance of the device it was run on. Since achieved compute throughput and/or memory bandwidth is below 60.0% of peak, the kernel is latency bound. The tool suggests looking at the *Scheduler Statistics* and *Warp State Statistics* for possible reasons.

<!--<img src="images/jacobi_5_sol.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1vo9J4LlT9qgOpZaiJ-joyi48gvZwS9Fa">


The *Scheduler Statistics* shows the activity summary of the schedulers issuing instructions. Every scheduler can issue one instruction per cycle, but for this kernel, each scheduler only issues an instruction every ~1287 cycles. This causes underutilization of the hardware resources and leads to less than optimal performance.


<!--<img src="/home/images/jacobi_5_schedule.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1f-FIk-7rret5LDdjDIZRCPTKjop6Tr1j">


Check the *Warp State Statistics* section  which presents an analysis  of the states of all warps  that spent cycles during the kernel execution. The warp states describe a warp's readiness or inability to issue its next instruction. As seen in the screenshot below, the most important stall reason is *LG (local/global) Throttle* which indicates extremely frequent memory instructions, according to the guided analysis rule (hover your mouse over each to see the description).

<!--<img src="images/jacobi_5_warp.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1qDGuDwFHva1px_74l_kvepnA4QqV3-4S">


When we check the *Source Counters* section for the top stall locations in your source based on sampling data, we can see the kernel has uncoalesced global accesses (note: you can click on the links and go to the line where these occur).

<!--<img src="images/jacobi_5_source.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1KHiijrNKvUVD0DPGDj7qNpujE-LPOUT0">


Last, looking at the *Occupancy* section, we achieved 71% occupancy which is the ratio of the number of active warps per multiprocessor to the maximum number of possible active warps. In other words, we are giving enough work to the device to keep it busy. So, the low memory throughput is a result of poor memory access patterns.

<!--<img src="images/jacobi_5_occ.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1Ca50hodiUnNGJaW3rRqe1iNc7VPDx7aj">

To conclude, the reason for low memory throughput is poor memory access patterns which means the memory layout of our arrays does not map well with the threads (read more at [Analysis-Driven Optimization: Analyzing and Improving Performance with NVIDIA Nsight Compute](https://developer.nvidia.com/blog/analysis-driven-optimization-analyzing-and-improving-performance-with-nvidia-nsight-compute-part-2/)). Now,  profile the `swap_data()` kernel with Nsight Compute.

In [30]:
!cd /home/source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:swap_data -f -o jacobi_step5_1 --set full ./jacobi_step5

==PROF== Connected to process 13464 (/home/source_code/lab6/jacobi_step5)
Error after iteration 0 = 0.00781058
==PROF== Profiling "swap_data": 0%....50%....100% - 31 passes
Error after iteration 25 = 0.000608074
Error after iteration 50 = 0.000366184
Error after iteration 75 = 0.000271309
Error after iteration 100 = 0.000219121
Error after iteration 125 = 0.000185579
Error after iteration 150 = 0.000161997
Error after iteration 175 = 0.000144383
Error after iteration 200 = 0.000130677
Error after iteration 225 = 0.000119665
Error after iteration 250 = 0.000110599
Error after iteration 275 = 0.000102984
Success!
Run time = 3.777 seconds
==PROF== Disconnected from process 13464
==PROF== Report: /home/source_code/lab6/jacobi_step5_1.ncu-rep


Download the report by running the below cell and open the report via the Nsight Compute user interface (UI).

In [31]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step5_1.ncu-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

According to the *GPU Speed of Light Throughput* section (see screenshot below), the memory is more heavily utilized than compute and we need to look at the *Memory Workload Analysis* to identify the L2 bottleneck.

<!--<img src="images/jacobi_swap_sol.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1kf2IowIQ1K8q4VANTMHjY5i3wrFv9jlz">


The *Memory Workload Analysis* section tells us that the kernel has a poor memory access pattern. Rather than accessing 4 bytes per thread per memory request, there are `32x32 = 1023` bytes of cache data transfers per request which is 8 times more sector loads.

<!--<img src="images/jacobi_swap_mem.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1rdEQQtRCfiBgdHw_RlIzBkXDL9b5Cc-9">


Compare the indexing scheme we used to the threading scheme. In a 2D threadblock, the `x` dimension is the contiguous dimension and the `y` dimension is the strided dimension. So, in a 32x32 thread block, each warp comprises of 32 threads in the `x` dimension and the `threadIdx.y` counts/enumerates 32 warps. To fix the uncoalessed memory access, we change `#define IDX(i, j) ((j) + (i) * N)` to `#define IDX(i, j) ((i) + (j) * N)`.

**Understand and analyze** the code:

In [32]:
!cat /home/source_code/lab6/jacobi_step6.cpp

// Copyright (c) 2022 NVIDIA Corporation.  All rights reserved.

#include <iostream>
#include <iomanip>
#include <cmath>
#include <limits>
#include <ctime>
#include <nvToolsExt.h>

#define N 2048

#define IDX(i, j) ((i) + (j) * N)

// error checking macro
#define cudaCheckErrors(msg)                                    \
    do {                                                        \
        cudaError_t __err = cudaGetLastError();                 \
        if (__err != cudaSuccess) {                             \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n",  \
                    msg, cudaGetErrorString(__err),             \
                    __FILE__, __LINE__);                        \
            fprintf(stderr, "*** FAILED - ABORTING\n");         \
            exit(1);                                            \
        }                                                       \
    } while (0)

void allocate_memory (float** f, float** f_old, float** error) {
  

Now, compile and profile the code.

In [33]:
!cd /home/source_code/lab6 && make jacobi_step6 && nsys profile --stats=true -o jacobi_step6 -f true ./jacobi_step6

nvcc -o jacobi_step6 -x cu -lnvToolsExt -lineinfo jacobi_step6.cpp
Error after iteration 0 = 0.00781058
Error after iteration 25 = 0.000608074
Error after iteration 50 = 0.000366183
Error after iteration 75 = 0.000271311
Error after iteration 100 = 0.000219121
Error after iteration 125 = 0.00018558
Error after iteration 150 = 0.000161997
Error after iteration 175 = 0.000144383
Error after iteration 200 = 0.000130678
Error after iteration 225 = 0.000119665
Error after iteration 250 = 0.000110599
Error after iteration 275 = 0.000102984
Success!
Run time = 3.164 seconds
Generating '/tmp/nsys-report-4494.qdstrm'
[1/8] [========================100%] jacobi_step6.nsys-rep
Importer error status: Importation succeeded with non-fatal errors.
**** Analysis failed with:
Status: TargetProfilingFailed
Props {
  Items {
    Type: DeviceId
    Value: "Local (CLI)"
  }
}
Error {
  Type: RuntimeError
  SubError {
    Type: ProcessEventsError
    Props {
      Items {
        Type: ErrorText
        Val

Download the report by running the below cell and open the report via the Nsight Systems user interface (UI).

In [ ]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step6.nsys-rep')

We can clearly see we made some overall improvements and reduced the total execution time.

Lets profile the `swap_data` kernel with Nsight Compute to see if we fixed the global memory access pattern.

In [34]:
!cd /home/source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:swap_data -f -o jacobi_step6_1 --set full ./jacobi_step6

==PROF== Connected to process 14829 (/home/source_code/lab6/jacobi_step6)
Error after iteration 0 = 0.00781058
==PROF== Profiling "swap_data": 0%....50%....100% - 31 passes
Error after iteration 25 = 0.000608074
Error after iteration 50 = 0.000366183
Error after iteration 75 = 0.000271309
Error after iteration 100 = 0.000219121
Error after iteration 125 = 0.000185581
Error after iteration 150 = 0.000161997
Error after iteration 175 = 0.000144383
Error after iteration 200 = 0.000130677
Error after iteration 225 = 0.000119665
Error after iteration 250 = 0.000110599
Error after iteration 275 = 0.000102984
Success!
Run time = 3.556 seconds
==PROF== Disconnected from process 14829
==PROF== Report: /home/source_code/lab6/jacobi_step6_1.ncu-rep


Download the report by running the below cell and open the report via the Nsight Compute UI.

In [36]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step6_1.ncu-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

When comparing the `swap_data` kernel with the previous step, we can see that we fixed the memory access pattern and improved the overall performance.

<!--<img src="images/jacobi_6_swap_base.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=14w5rrFjyplbRfxsHysPc-vyOIlzR_G0w">


We are now latency bound and if you look at the *Source Counters* section, we no longer have any non-coalesced global accesses. Now,  profile the `jacobi_step` kernel with Nsight Compute.

In [37]:
!cd /home/source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:jacobi -f -o jacobi_step6_0 --set full --import-source 1 ./jacobi_step6

==PROF== Connected to process 15205 (/home/source_code/lab6/jacobi_step6)
Error after iteration 0 = 0.00781058
==PROF== Profiling "jacobi_step": 0%....50%....100% - 31 passes
Error after iteration 25 = 0.000608074
Error after iteration 50 = 0.000366183
Error after iteration 75 = 0.00027131
Error after iteration 100 = 0.000219121
Error after iteration 125 = 0.000185581
Error after iteration 150 = 0.000161998
Error after iteration 175 = 0.000144383
Error after iteration 200 = 0.000130677
Error after iteration 225 = 0.000119665
Error after iteration 250 = 0.000110599
Error after iteration 275 = 0.000102984
Success!
Run time = 3.854 seconds
==PROF== Disconnected from process 15205
==PROF== Report: /home/source_code/lab6/jacobi_step6_0.ncu-rep


Download the report by running the below cell and open the report via the Nsight Compute UI.

In [ ]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step6_0.ncu-rep')

Compared to the previous version of the code, the `jacobi_step` kernel did achieve a substantial improvement.

<!--<img src="images/jacobi_6_jacobi_base.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1gpb4rnPbHLGQGRfJLGq97hijy9xvygxU">


Looking at the *Scheduler Statistics*, we see 99.9% of the cycles have no warps eligible to issue work. On cycles with no eligible warps, the issue slot is skipped and no instruction is issued. Having many skipped issue slots indicates poor latency hiding.

<!--<img src="images/jacobi_6_jacobi_schedule.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=11wJJbpZmgPVL9c7n7_0NDPeUG2qx7i8m">


The reason for this is the atomic update to the error counter inside the `jacobi_step()` kernel (`atomicAdd(error, df * df)`). If multiple threads write to the same location at the same time, they will serialize and stall. We can refactor the kernel in a way to use a more efficient reduction scheme that uses fewer overall atomics.


**Understand and analyze** the code:

In [39]:
!cat /home/source_code/lab6/jacobi_step7.cpp

// Copyright (c) 2022 NVIDIA Corporation.  All rights reserved.

#include <iostream>
#include <iomanip>
#include <cmath>
#include <limits>
#include <ctime>
#include <nvToolsExt.h>

#define N 2048

#define IDX(i, j) ((i) + (j) * N)

// error checking macro
#define cudaCheckErrors(msg)                                    \
    do {                                                        \
        cudaError_t __err = cudaGetLastError();                 \
        if (__err != cudaSuccess) {                             \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n",  \
                    msg, cudaGetErrorString(__err),             \
                    __FILE__, __LINE__);                        \
            fprintf(stderr, "*** FAILED - ABORTING\n");         \
            exit(1);                                            \
        }                                                       \
    } while (0)

void allocate_memory (float** f, float** f_old, float** error) {
  

Now, compile and profile the code.

In [40]:
!cd /home/source_code/lab6 && make jacobi_step7 && nsys profile --stats=true -o jacobi_step7 -f true ./jacobi_step7

nvcc -o jacobi_step7 -x cu -lnvToolsExt -lineinfo jacobi_step7.cpp
Error after iteration 0 = 0.00781059
Error after iteration 25 = 0.000608098
Error after iteration 50 = 0.000366206
Error after iteration 75 = 0.000271333
Error after iteration 100 = 0.000219136
Error after iteration 125 = 0.000185597
Error after iteration 150 = 0.000162008
Error after iteration 175 = 0.000144401
Error after iteration 200 = 0.000130694
Error after iteration 225 = 0.00011968
Error after iteration 250 = 0.000110613
Error after iteration 275 = 0.000103
Success!
Run time = 0.7984 seconds
Generating '/tmp/nsys-report-e56a.qdstrm'
[1/8] [========================100%] jacobi_step7.nsys-rep
Importer error status: Importation succeeded with non-fatal errors.
**** Analysis failed with:
Status: TargetProfilingFailed
Props {
  Items {
    Type: DeviceId
    Value: "Local (CLI)"
  }
}
Error {
  Type: RuntimeError
  SubError {
    Type: ProcessEventsError
    Props {
      Items {
        Type: ErrorText
        Value

Download the report by running the below cell and open the report via the Nsight Systems UI.

In [41]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step7.nsys-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We managed to reduce the total execution time enormously. Profile the `jacobi_step()` kernel with Nsight Compute to investigate more.

In [42]:
!cd /home/source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:jacobi -f -o jacobi_step7_0 --set full --import-source 1 ./jacobi_step7

==PROF== Connected to process 16182 (/home/source_code/lab6/jacobi_step7)
Error after iteration 0 = 0.00781059
==PROF== Profiling "jacobi_step": 0%....50%....100% - 31 passes
Error after iteration 25 = 0.000608098
Error after iteration 50 = 0.000366206
Error after iteration 75 = 0.000271333
Error after iteration 100 = 0.000219136
Error after iteration 125 = 0.000185597
Error after iteration 150 = 0.000162008
Error after iteration 175 = 0.000144401
Error after iteration 200 = 0.000130694
Error after iteration 225 = 0.00011968
Error after iteration 250 = 0.000110613
Error after iteration 275 = 0.000103
Success!
Run time = 1.268 seconds
==PROF== Disconnected from process 16182
==PROF== Report: /home/source_code/lab6/jacobi_step7_0.ncu-rep


Download the report by running the below cell and open the report via the Nsight Compute UI.

In [43]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step7_0.ncu-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<!--<img src="images/jacobi_7_sol.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1MxvgMKvTi-AdN_ijeoh9Fgwu8MqgOIM8">


The *GPU Speed of Light* section shows we increase both compute and memory utilization compared to the previous section. Moreover, the Roofline charts shows an improvement in performance and an increase in arithmetic intensity.

<!--<img src="images/jacobi_7_roof.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1Hp8zLPe2pK8L6_2gAHcmjFEVFR1ueuLH">


Comparing the `jacobi_step` and `swap_data` kernel, one has more dynamic random-access memory (DRAM) throughput than the `jacobi_step` kernel.

<!--<img src="images/jacobi_7_base.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=19U4_i56GV85iCr9EPQmFfklaLC50fS-0">


### Step 7: Shared Memory
Although the code runs faster, there are still uncoalesced accesses that we need to address. The issue with the stencil operations is the cache reuse and as the array (problem size) grows, due to the stride between row `j` and row `j+1` , there will be less cache reuse. To solve this issue, we can read the data from the shared memory which is closer to the GPU cores. This will result in accessing the DRAM less. We implement the simplest version for ease but this might not be the best solution. To learn more on how to understand and optimize shared memory accesses using Nsight Compute, please watch the [GTC talk](https://www.nvidia.com/en-us/on-demand/session/gtcspring22-s41723/).


**Understand and analyze** the code:

In [ ]:
!cat /home/source_code/lab6/jacobi_step8.cpp

Now, compile and profile the code.

In [44]:
!cd /home/source_code/lab6 && make jacobi_step8 && nsys profile --stats=true -o jacobi_step8 -f true ./jacobi_step8

nvcc -o jacobi_step8 -x cu -lnvToolsExt -lineinfo jacobi_step8.cpp
Error after iteration 0 = 0.00781059
Error after iteration 25 = 0.000608098
Error after iteration 50 = 0.000366206
Error after iteration 75 = 0.000271333
Error after iteration 100 = 0.000219136
Error after iteration 125 = 0.000185597
Error after iteration 150 = 0.000162008
Error after iteration 175 = 0.000144401
Error after iteration 200 = 0.000130694
Error after iteration 225 = 0.00011968
Error after iteration 250 = 0.000110613
Error after iteration 275 = 0.000103
Success!
Run time = 0.7491 seconds
Generating '/tmp/nsys-report-f8d5.qdstrm'
[1/8] [========================100%] jacobi_step8.nsys-rep
Importer error status: Importation succeeded with non-fatal errors.
**** Analysis failed with:
Status: TargetProfilingFailed
Props {
  Items {
    Type: DeviceId
    Value: "Local (CLI)"
  }
}
Error {
  Type: RuntimeError
  SubError {
    Type: ProcessEventsError
    Props {
      Items {
        Type: ErrorText
        Value

Download the report by running the below cell and open the report via the Nsight Systems UI.

In [45]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step8.nsys-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Profile the kernel with Nsight Compute and inspect it further.

In [46]:
!cd /home/source_code/lab6 && ncu --launch-count 1 --launch-skip 5 -k --regex:jacobi --export jacobi_step8_0 --force-overwrite --set full ./jacobi_step8

==PROF== Connected to process 17286 (/home/source_code/lab6/jacobi_step8)
Error after iteration 0 = 0.00781059
==PROF== Profiling "jacobi_step": 0%....50%....100% - 31 passes
Error after iteration 25 = 0.000608098
Error after iteration 50 = 0.000366206
Error after iteration 75 = 0.000271333
Error after iteration 100 = 0.000219136
Error after iteration 125 = 0.000185597
Error after iteration 150 = 0.000162008
Error after iteration 175 = 0.000144401
Error after iteration 200 = 0.000130694
Error after iteration 225 = 0.00011968
Error after iteration 250 = 0.000110613
Error after iteration 275 = 0.000103
Success!
Run time = 1.174 seconds
==PROF== Disconnected from process 17286
==PROF== Report: /home/source_code/lab6/jacobi_step8_0.ncu-rep


Download the report by running the below cell and open the report via the Nsight Compute UI.

In [47]:
from google.colab import files
files.download('/home/source_code/lab6/jacobi_step8_0.ncu-rep')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


<!--<img src="images/jacobi_8_sol.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1GL_RQTscRUg0CuXQI-aUpK_kuJiGemvr">


We achieved better compute and memory utilization compared to the previous section and memory access patterns are fully coalesced.


<!--<img src="images/jacobi_8_source.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=1mcjFV8X_BHHum8HmhNyW35hrukFsUIY5">

The Nsight Systems profiler report shows how fast the kernels are. However, take to account that you might have to increase the problem size again as once again the device warmup might be a performance limiter.

<!--<img src="images/jacobi_8_nsight.png" width="80%">-->
<img src="https://drive.google.com/uc?export=download&id=11zMfgCJIMOoJs_RZcq6nkG_KZKskPTDW">

# Advanced Topics

## Learning objectives
The **goal** of this lab is to:

- Learn about tracing of Message Passing Interface (MPI), OpenSHMEM, NVSHMEM™ and NVIDIA Collective Communication Library (NCCL)
- Learn how to do multi-process profiling

We do not intend to cover:

- How to use NVSHMEM, NCCL, and MPI

### NVIDIA Nsight Systems
NVIDIA Nsight™ Systems tool offers system-wide performance analysis to visualize the application’s algorithms, help identify optimization opportunities, and improve the performance of applications running on a system consisting of multiple CPUs and GPUs. Nsight Systems is packed with many features. A few of the features highlighted in the above screenshot:

- Application programming interface (API) tracing of CUDA libraries and deep learning frameworks
- CPU utilization, CPU thread states and thread migration as well as CPU callstack sampling
- Operating system (OS) runtime library calls
- GPU activities (kernels and memory copies) as well as GPU metrics

This section briefly explores other features of Nsight Systems that were not covered as part of the other labs.

#### GPU Metrics Sampling

Nsight Systems has a GPU Metrics feature that is used to identify performance limiters in applications using GPU for computations. It uses periodic sampling to gather performance metrics and detailed timing statistics associated with different GPU hardware units taking advantage of specialized hardware to capture this data in a single pass with minimal overhead. These metrics provide an overview of GPU efficiency over time within compute and input/output (I/O) activities:

- I/O throughputs: PCIe, NVLink, and dynamic random access memory (DRAM)
- Streaming Multiprocessor / Shared Processor (SM) utilization: SMs activity, Tensor Core activity, instructions issued, warp occupancy (including unallocated slots)

These metrics can also help users answer the common questions:

- Is my GPU idle?
- Is my instruction rate low (possibly IO bound)?
- Is my GPU full? Sufficient kernel grids size and streams? Are my SMs and warp slots full?
- Can I see GPU Direct Remote Direct Memory Acccess (RDMA)/Storage or other transfers?
- Am I using TensorCores?
- Am I possibly blocked on I/O, or number of warps, etc?

Nsight Sytems GPU Metrics require NVIDIA GPUs with the Turing™ architecture or newer. Learn more about GPU metrics at https://docs.nvidia.com/nsight-systems/UserGuide/index.html#gpu-metric-sampling

<!--<img src="images/gpu_metrics.png">-->
<img src="https://drive.google.com/uc?export=download&id=1F9kGVB3KHOsSGvfgsxDOeKKzE5K0U_2O">


#### TRACING OF MPI, OpenSHMEM, NVSHMEM AND NCCL

Nsight Systems supports MPI as well as OpenSHMEM tracing. You can record MPI communication parameters and track the MPI communicators. So, if you want to follow the data and see which MPI ranks are communicating with each other and how much data is transferred, you can see this information in the report.

<!--<img src="images/mpi_comm.png">-->
<img src="https://drive.google.com/uc?export=download&id=1wBtIVybtVjPGFlMcl16WfI9iXcT9Bm5L">


In the above screenshot you see the tooltip for an `MPI_Irecv` (bottom right) with its MPI tag, the number of bytes that have been received, the sender of the data, and also the communicator. In the case of MPI, you can also trace MPI for Fortran applications.

Besides MPI and OpenSHMEM, where we intercept the library calls, Nsight Systems can also trace calls into NVSHMEM and NCCL libraries based on NVIDIA Tools Extension SDK (NVTX) as explained in previous labs. The result looks similar to what is shown in the above screenshot, with the function names of the respective API and the respective row labels, e.g. NCCL and NVSHMEM rows instead of MPI and UCX as in the above screenshot.

In the screenshot, you see the execution timeline of a super short range of an MPI program which triggers some `MPI_Isends` and `MPI_Irecvs`. For each MPI call you get the communication parameters and you also get the Unified Communication X (UCX) API calls, which are triggered by the MPI implementation, which in this example is OpenMPI.


#### UCX API TRACING

The UCX layer is an open-source communication framework that acts as a common library and API for several higher-level communication libraries, for example Open MPI (including its OpenSHMEM implementation) and MPICH. If UCX library trace is selected Nsight Systems will trace the subset of functions of the UC Protocol (UDP) layer that are most likely involved in performance bottlenecks. If OpenSHMEM library trace is selected, Nsight Systems will trace the subset of OpenSHMEM API functions that are most likely involved in performance bottlenecks.

<!--<img src="images/ucx.png">-->
<img src="https://drive.google.com/uc?export=download&id=19BClqtSEB0pYJ4ghp6cvKA_h1vAWc3i5">


In the above screenshot, we have both `MPI_Isend` and `MPI_Irecv` calls that trigger UCP API calls (you only see the `MPI_Isend` calls, because the `MPI_Irecv` calls are super short for this particular example). The bottom row in the timeline shows the processing of transfers from non-blocking UCP communication operations. In the UCX row, you see the submit functions and in the row below you see when the processing of the transfers starts (if we were to zoom out, we could also see when the processing ends).

#### NIC Performance Metrics
NVIDIA ConnectX® smart network interface cards (smart NICs) offer advanced hardware offloads and accelerations for network operations. Viewing smart NICs metrics on Nsight Systems timeline enables developers to better understand their application’s network usage and use this information to optimize the application’s performance.

<!--<img src="images/NIC.png"> -->
<img src="https://drive.google.com/uc?export=download&id=1gq1GKYx71JxSlnUeaL7F-US4rmj5E_Qs">


The performance counters are displayed over the Nsight Systems timeline, letting you know when the application is sending and receiving data. There are also counters that indicate network congestion like the `IB Send Wait` counter that you see in the above screenshot.

### Nsight Systems Multi-Process Profiling

On compute clusters where you have to use a workload manager or want to do a run over multiple nodes, the `nsys profile` command is prefixed before the application. With this, a report file is generated for each process. If you can launch your application without a workload manager on a single node (e.g. with `mpirun`), you can prefix `nsys profile` before `mpirun` and a single report including all processes is generated.

- **Single Node**: `nsys profile [nsys_args] mpirun [mpirun_args] your_executable`. The command will create one report file.
- **Multiple Nodes**: `mpirun [mpirun_args] nsys profile [nsys_args] your_executable`, you can set output report name with `-o report_name_%q{OMPI_COMM_WORLD_RANK}`. (For OpenMPI, PMI_RANK for MPICH and SLURM_PROCID for Slurm). The command will create one report file per MPI rank.

You can also profile only specific ranks:

```
#!/bin/bash
# OMPI_COMM_WORLD_LOCAL_RANK for node local rank
if [ $OMPI_COMM_WORLD_RANK -eq 0 ]; then
    nsys profile -t mpi "$@"
else
    "$@"
fi
```

Below is an example command that was run on a compute facility that uses the *SLURM* workload manager using two nodes.

```
srun [SRUN_ARGS] nsys profile -t mpi,ucx -s none --nic-metrics=true -o ./report_mpi.%q{SLURM_PROCID} -f true ./myprogram [PROGRAM_ARGS]
```

where,

- `nsys profile`: starts a profiling session
- `-t, --trace=...`: sets the APIs to be traced, in this example, it is UCX and MPI
- `-s,--sample=[cpu|none]`: controls CPU IP sampling
- `--nic-metrics=[true|false]`: controls Network Interface Cards (NIC) metrics collection
- `-o, --output=report#`: output profile report file path; and
- `-f --force-overwrite`: overwrite the output report file, if it already exists

To learn more about other switches to use with `nsys profile`, you can type `nsys profile --help` on the command line or read the [online docs](https://docs.nvidia.com/nsight-systems/UserGuide/index.html#cli-profiling).

### Nsight Compute Multi-Process Profiling

On a single-node submission, one Nsight Compute instance can profile all launched child processes with the data for all processes stored in one report file.

`ncu --target-processes all -o <singlereport-name> <app> <args>`

On multi-node submissions, one tool instance can be used per node. Make sure instances don’t write to the same report file on a shared disk.

`ncu -o report_%q{OMPI_COMM_WORLD_RANK} <app> <args>`

Similar to Nsight Systems, consider profiling only a single rank, for example using a wrapper script (see below example)

```
#!/bin/bash
if [[ "$OMPI_COMM_WORLD_RANK" == "3" ]] ; then
    /sw/cluster/cuda/11.1/ nsight-compute/ncu -o report_${OMPI_COMM_WORLD_RANK} --target-processes all $*
else
    $*
fi
```

# Links and Resources


[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA Nsight Compute](https://docs.nvidia.com/nsight-compute/index.html)

[CUDA Toolkit Documentation](https://docs.nvidia.com/cuda/)

**NOTE**: To be able to see the Nsight Systems and Compute profiler outputs, please download the latest versions from the below pages:

- https://developer.nvidia.com/nsight-systems
- https://developer.nvidia.com/nsight-compute


Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---

## Licensing

Copyright © 2022 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.